# Problem set: Tensorflow
> Tangqi Feng

These problems relate to the Python package [Tensorflow](https://www.tensorflow.org/).
We will again use the famous [iris data set](https://en.wikipedia.org/wiki/Iris_flower_data_set).


# Load and convert data

In [1]:
# adapt from https://github.com/antonrufino/TensorFlow-IrisNN/blob/master/iris_nn.py
# inport numpy
import numpy as np
# load Iris data set as str,because the 'species' column's type is str
OriginalData = np.loadtxt("iris.csv",str, delimiter=",", skiprows=1, unpack=True)
iris = OriginalData.transpose()

# the data stored like this:
for i in range(5):
    print(iris[i,:])

['5.1' '3.5' '1.4' '0.2' 'setosa']
['4.9' '3' '1.4' '0.2' 'setosa']
['4.7' '3.2' '1.3' '0.2' 'setosa']
['4.6' '3.1' '1.5' '0.2' 'setosa']
['5' '3.6' '1.4' '0.2' 'setosa']


## 1. Split the data into training and testing
Split the data set into a training set and a testing set.
You should investigate the best way to do this, and list any online references used in your notebook.
If you wish to, you can write some code to randomly separate the data on the fly.


In [2]:
# Randomly split the data into training and testing
# Adapt from : https://stackoverflow.com/questions/17412439/how-to-split-data-into-trainset-and-testset-randomly
np.random.shuffle(iris)
# define 100 set of data for training, and 50 for test
train, test = iris[:100], iris[100:]
# get input data (sepal_length, sepal_width, petal_length, petal_width)
in_train = train[:,:4]
in_test = test[:,:4]
# get output data
out_train = train[:,4]
out_test = test[:,4]
# convert inputs data from type(str) to float
in_train = np.array(in_train).astype(np.float)
in_test = np.array(in_test).astype(np.float)
# get a list of output categories as 'one-hot vectors'
# a one-hot vector is a vector which is 0 in most dimensions, and 1 in a single dimension
# in this case,convert like this:
#    setosa      ->[1,0,0]
#    virginica   ->[0,1,0]
#    versicolor  ->[0,0,1]
# np.unique --> return a unique list in array, 
# 'return_inverse=True' will return the index number of such unique element
# more details: https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.unique.html
uniq_test, ids_test = np.unique(out_test, return_inverse=True)
uniq_train, ids_train = np.unique(out_train, return_inverse=True)
# np_utils.to_categorical --> Converts a class vector (integers) to binary class matrix.
# more details: https://keras.io/utils/
from keras.utils import np_utils
onehot_train = np_utils.to_categorical(ids_train, len(uniq_train))
onehot_test = np_utils.to_categorical(ids_test, len(uniq_test))
# after np.unique & np_utils.to_categorical two operation
# the data like this: 
for i in range(5):
    print('species: '+(str)(out_test[i])+' --> '+(str)(ids_test[i])\
          +' --> '+(str)(onehot_test[i,:]))

Using TensorFlow backend.


species: virginica --> 2 --> [ 0.  0.  1.]
species: setosa --> 0 --> [ 1.  0.  0.]
species: versicolor --> 1 --> [ 0.  1.  0.]
species: virginica --> 2 --> [ 0.  0.  1.]
species: versicolor --> 1 --> [ 0.  1.  0.]


## 2. Use Tensorflow to create model
Use Tensorflow to create a model to predict the species of Iris from a flower's sepal width, sepal length, petal width, and petal length.

In [3]:
# a set of data contains: sepal_length, sepal_width, petal_length, petal_width and species
# import tensorflow
import tensorflow as tf
# create a model:
# input layer
# input_data [None,4] means:None: do not deside the number of set of input numbers
#                           4: (sepal_length, sepal_width, petal_length, petal_width)
x = tf.placeholder(tf.float32,[None,4])
# output_data (species) means:None: do not deside the number of set of input numbers
#                           3: species:(setosa,versicolor,virginica)
y = tf.placeholder(tf.float32,[None,3])
# output layer
# tf.truncated_normal --> outputs random values from a truncated normal distribution
# 'stddev' is the standard deviation of the truncated normal distribution.
# method details: https://www.tensorflow.org/api_docs/python/tf/truncated_normal
# here, designed a [4,3] matrix for Weight(W), simply means 4 data inputs and 3 outputs
W = tf.Variable(tf.truncated_normal([4,3],stddev=0.1))
# set bias for output value, because output 3 values, so the size of bias is [3]
b = tf.Variable(tf.zeros([3]) + 1)
# tf.matmul --> a method multiplies matrix
# tf.nn.softmax --> softmax gives us a list of values between 0 and 1 that add up to 1
prediction = tf.nn.softmax(tf.matmul(x,W) + b)

## 3. Train the model
Use the training set to train your model.

In [4]:
# here, use cross_entropy and softmax method:
# cross_entropy function --> as the loss function
# softmax is the thing to do, because softmax gives us a list of values between 0 and 1 that add up to 1
# tf.nn.softmax_cross_entropy_with_logits method combines cross_entropy and softmax
# more details: https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
# use GradientDescent optimizer to train
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
# initial glabal variables
init = tf.global_variables_initializer()
# create loop to train
with tf.Session() as sess:
    sess.run(init)
    for eposh in range(100):    # times for training all training)set
        sess.run(train_step,{x:in_train, y:onehot_train})

## 4. Test the model
Use the testing set to test your model, clearly calculating and displaying the error rate.

In [5]:
# calculate accuracy
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) # correct return true, otherwise return false
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) # true->1.0   false->0
# create loop to test
with tf.Session() as sess:
    sess.run(init)
    for eposh in range(101):    # times for training all training)set
        sess.run(train_step,{x:in_train, y:onehot_train})                  #train               
        acc = sess.run(accuracy,{x:in_test, y:onehot_test})                #test
        if(eposh%10==0):
            print("Iter " + str(eposh) + ",Testing Accuracy " + str(acc) + ",Testing Error Rate " + str(1-acc))


Iter 0,Testing Accuracy 0.38,Testing Error Rate 0.620000004768
Iter 10,Testing Accuracy 0.66,Testing Error Rate 0.339999973774
Iter 20,Testing Accuracy 0.66,Testing Error Rate 0.339999973774
Iter 30,Testing Accuracy 0.66,Testing Error Rate 0.339999973774
Iter 40,Testing Accuracy 0.72,Testing Error Rate 0.27999997139
Iter 50,Testing Accuracy 0.84,Testing Error Rate 0.160000026226
Iter 60,Testing Accuracy 0.88,Testing Error Rate 0.120000004768
Iter 70,Testing Accuracy 0.9,Testing Error Rate 0.100000023842
Iter 80,Testing Accuracy 0.92,Testing Error Rate 0.0799999833107
Iter 90,Testing Accuracy 0.94,Testing Error Rate 0.0600000023842
Iter 100,Testing Accuracy 0.98,Testing Error Rate 0.0199999809265


# Analysis
## here we get 98% accuracy
## change some stuff, will get different result
## * initial value, here we use 'tf.truncated_normal' function for Weight and 'tf.zeros' for bias
## * loss calculate method, here we use 'cross_entropy' function
## * activation method, here we use 'softmax' function
## * optimizer, here we use GradientDescent optimizer, learning_rate (0.2)
## * training, testing data set, here we have 100 training data set and 50 test data set
## * times for training, here we trains 101 times
## * neural network layer, here we just use a input layer (4 nodes) and a output layer (3 nodes). It is possible to add some middle layer.
    for example, input layer (4 nodes) -> middle layer (X nodes) -> middle layer (Y nodes) ........ ->output layer (3 nodes)

# End